In [1]:
import os
import requests
import torch
import pickle
from PIL import Image
from torch.utils.data import DataLoader
from tqdm import tqdm # Ensure tqdm is imported
from transformers import BlipProcessor, BlipForQuestionAnswering, BitsAndBytesConfig
from datasets import load_dataset
import gc
from peft import LoraConfig, get_peft_model

2025-05-15 21:06:08.869891: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747343169.067854      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747343169.122489      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:

# --- Configuration ---
MODEL_NAME = "Salesforce/blip-vqa-base"
CSV_PATH = r"/kaggle/input/merged-training/merged_final.csv"
IMAGES_BASE_FOLDER = r"/kaggle/input/vrdatasets/abo-images-small"
OUTPUT_MODEL_DIR = "/kaggle/working/blip-finetuned"
TRACKING_FILE = "tracking_information.pkl"

R_LORA = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05
# Updated TARGET_MODULES for BLIP. Common ones are query/key/value in attention layers.
# The exact names depend on the model architecture.
# You might need to inspect model.named_modules() to find precise names if these don't work.
# Common patterns for BLIP-like models (BERT-based text parts, ViT-based vision parts):
TARGET_MODULES = ["q_proj",
        "k_proj",
        "v_proj",
        "mlp.fc1",
        "mlp.fc2",
        "output.dense"]


IMAGE_SIZE = (384, 384)
QUESTION_MAX_LENGTH = 36
ANSWER_MAX_LENGTH = 16 # Increased from 8

BATCH_SIZE = 16
NUM_WORKERS = 2
LEARNING_RATE = 5e-5
NUM_EPOCHS = 3
PATIENCE = 10
GRADIENT_ACCUMULATION_STEPS = 1
LOG_INTERVAL = 40  # Print loss every 50 steps/batches

LOAD_IN_FP16 = True
LOAD_IN_8BIT = False
LOAD_IN_4BIT = False

In [3]:
# --- Setup ---
if 'model' in globals(): del model
if 'processor' in globals(): del processor
if 'optimizer' in globals(): del optimizer
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"Initial GPU Memory: Allocated={torch.cuda.memory_allocated(0)/1024**2:.2f}MB, Reserved={torch.cuda.memory_reserved(0)/1024**2:.2f}MB")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"CUDA Device Name: {torch.cuda.get_device_name(0)}")

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

processor = BlipProcessor.from_pretrained(MODEL_NAME)
peft_config = LoraConfig(
    r=R_LORA,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none"
)

model_kwargs = {}
if torch.cuda.is_available():
    if LOAD_IN_FP16 and not (LOAD_IN_8BIT or LOAD_IN_4BIT):
        print("Loading model in float16...")
        model_kwargs["torch_dtype"] = torch.float16
    elif LOAD_IN_8BIT:
        print("Loading model in 8-bit...")
        quantization_config = BitsAndBytesConfig(load_in_8bit=True)
        model_kwargs["quantization_config"] = quantization_config
    elif LOAD_IN_4BIT:
        print("Loading model in 4-bit...")
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
            bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4",
        )
        model_kwargs["quantization_config"] = quantization_config
    else: print("Loading model in float32...")
else: print("Loading model on CPU in float32...")

model = BlipForQuestionAnswering.from_pretrained(MODEL_NAME, **model_kwargs)
print(f"Base model '{MODEL_NAME}' loaded.")

# --- (Optional) Calculate parameters of the BASE model before LoRA ---
base_total_params = sum(p.numel() for p in model.parameters())
base_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n--- Base Model Parameters (Before LoRA) ---")
print(f"Total parameters: {base_total_params:,}")
print(f"Initially trainable parameters: {base_trainable_params:,}\n")

print("Applying PEFT LoRA adapters...")
model = get_peft_model(model, peft_config)

print("\n--- Model Parameters AFTER LoRA ---")
model.print_trainable_parameters()
total_params_after_lora = sum(p.numel() for p in model.parameters())
trainable_params_after_lora = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters (including frozen base + LoRA adapters): {total_params_after_lora:,}")
print(f"Trainable LoRA parameters (manually calculated): {trainable_params_after_lora:,}")
if total_params_after_lora > 0:
    print(f"Percentage of trainable parameters in current model object: {(trainable_params_after_lora / total_params_after_lora) * 100:.4f}%\n")

print(f"Moving model to {device}...")
model.to(device)
print("Model moved to device.")

if torch.cuda.is_available():
    print(f"After model loading & .to(device): Allocated={torch.cuda.memory_allocated(0)/1024**2:.2f}MB, Reserved={torch.cuda.memory_reserved(0)/1024**2:.2f}MB")

if hasattr(torch, 'compile') and os.environ.get("KAGGLE_RUNTIME_VERSION", "0").startswith("Python"):
    if device.type == 'cuda':
        print("Compiling the model... (this may take a moment)")
        try:
            model = torch.compile(model, mode="reduce-overhead")
            print("Model compiled successfully.")
        except Exception as e:
            print(f"Could not compile model: {e}. Proceeding without compilation.")
    else: print("Model is on CPU, skipping torch.compile.")
else: print("torch.compile not available or not in a suitable Kaggle environment. Proceeding without compilation.")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Initial GPU Memory: Allocated=0.00MB, Reserved=0.00MB
Using device: cuda
CUDA Device Name: Tesla P100-PCIE-16GB


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Loading model in float16...


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

Base model 'Salesforce/blip-vqa-base' loaded.

--- Base Model Parameters (Before LoRA) ---
Total parameters: 384,672,572
Initially trainable parameters: 384,672,572

Applying PEFT LoRA adapters...

--- Model Parameters AFTER LoRA ---
trainable params: 4,128,768 || all params: 388,801,340 || trainable%: 1.0619
Total parameters (including frozen base + LoRA adapters): 388,801,340
Trainable LoRA parameters (manually calculated): 4,128,768
Percentage of trainable parameters in current model object: 1.0619%

Moving model to cuda...
Model moved to device.
After model loading & .to(device): Allocated=776.91MB, Reserved=798.00MB
torch.compile not available or not in a suitable Kaggle environment. Proceeding without compilation.


In [4]:

class VQADataset(torch.utils.data.Dataset):
    def __init__(self, dataset, processor, images_folder):
        self.dataset = dataset
        self.processor = processor
        self.images_folder = images_folder
    def __len__(self): return len(self.dataset)
    def __getitem__(self, idx):
        item = self.dataset[idx]
        question = item['question'] # Original question
        answer = item['answer']
        rel_path = item['full_image_path']
        
        instructed_question = f"{question} Answer in one word."

        image_path = os.path.join(self.images_folder, *rel_path.replace('\\', '/').split('/'))
        try:
            image = Image.open(image_path).convert("RGB").resize(IMAGE_SIZE)
        except FileNotFoundError: raise FileNotFoundError(f"Image not found: {image_path}")
        except Exception as e: raise Exception(f"Could not load image {image_path}: {e}")
        
        # Use the 'instructed_question' when processing
        encoding = self.processor(
            images=image, 
            text=instructed_question, # <--- USE MODIFIED QUESTION HERE
            padding="max_length", 
            truncation=True, 
            max_length=QUESTION_MAX_LENGTH, # Max length for the (question + instruction)
            return_tensors="pt"
        )
        encoding = {k: v.squeeze(0) for k, v in encoding.items()}

        labels = self.processor.tokenizer(
            text=answer, 
            max_length=ANSWER_MAX_LENGTH, 
            padding="max_length", 
            truncation=True, 
            return_tensors="pt"
        )["input_ids"].squeeze(0)
        encoding["labels"] = labels
        return encoding


In [5]:

print("Loading full dataset...")
full_ds = load_dataset("csv", data_files=CSV_PATH, split="train")
num_rows_to_keep = 10000000 # Increased from 50k
ds_subset = full_ds.select(range(len(full_ds) - min(num_rows_to_keep, len(full_ds)), len(full_ds))) if len(full_ds) > num_rows_to_keep else full_ds
print(f"Using subset of {len(ds_subset)} rows.")

split = ds_subset.shuffle(seed=42).train_test_split(test_size=0.1)
training_hf_dataset, valid_hf_dataset = split["train"], split["test"]
print(f"Split into: Training: {len(training_hf_dataset)}, Validation: {len(valid_hf_dataset)}")

def is_valid_example(example):
    # Ensure answer is a non-empty string and image path exists
    if not (isinstance(example.get('answer'), str) and example['answer'].strip() and example.get('full_image_path')):
        return False

    # Normalize the answer (strip and lowercase)
    ans = example['answer'].strip().lower()
    
    # Filter out yes/no questions
    return ans not in ["yes", "no"]
training_hf_dataset = training_hf_dataset.filter(is_valid_example)
valid_hf_dataset = valid_hf_dataset.filter(is_valid_example)
print(f"Filtered sizes: Training: {len(training_hf_dataset)}, Validation: {len(valid_hf_dataset)}")

if not training_hf_dataset or not valid_hf_dataset:
    print("Error: Training or validation dataset is empty. Exiting.")
    exit()

train_dataset = VQADataset(dataset=training_hf_dataset, processor=processor, images_folder=IMAGES_BASE_FOLDER)
valid_dataset = VQADataset(dataset=valid_hf_dataset, processor=processor, images_folder=IMAGES_BASE_FOLDER)

def collate_fn_safe(batch):
    batch = [item for item in batch if item is not None]
    return torch.utils.data.dataloader.default_collate(batch) if batch else None

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, collate_fn=collate_fn_safe)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, collate_fn=collate_fn_safe)

optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scaler_enabled = torch.cuda.is_available() and (LOAD_IN_FP16 or not (LOAD_IN_8BIT or LOAD_IN_4BIT))
scaler = torch.cuda.amp.GradScaler(enabled=scaler_enabled)
autocast_enabled = scaler_enabled
autocast_dtype = torch.float16 if LOAD_IN_FP16 and torch.cuda.is_available() else None
print(f"GradScaler enabled: {scaler.is_enabled()}, AMP Autocast enabled: {autocast_enabled}, dtype: {autocast_dtype}")

tracking_information = []
min_eval_loss = float("inf")
early_stop_counter = 0


Loading full dataset...


Generating train split: 0 examples [00:00, ? examples/s]

Using subset of 131847 rows.
Split into: Training: 118662, Validation: 13185


Filter:   0%|          | 0/118662 [00:00<?, ? examples/s]

Filter:   0%|          | 0/13185 [00:00<?, ? examples/s]

Filtered sizes: Training: 118622, Validation: 13180
GradScaler enabled: True, AMP Autocast enabled: True, dtype: torch.float16


/tmp/ipykernel_19/3290491606.py:42: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=scaler_enabled)


In [6]:
# loading checkpoints

start_epoch = 0
checkpoint_path = os.path.join(OUTPUT_MODEL_DIR, "checkpoint.pth")
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
    if scaler.is_enabled() and checkpoint["scaler_state_dict"]:
        scaler.load_state_dict(checkpoint["scaler_state_dict"])
    start_epoch = checkpoint["epoch"] + 1
    early_stop_counter = checkpoint["early_stop_counter"]
    min_eval_loss = checkpoint["min_eval_loss"]
    tracking_information = checkpoint["tracking_information"]
    print(f"Resumed training from epoch {start_epoch}")
else:
    print("Starting training from scratch.")


Starting training from scratch.


In [7]:

# --- Training Loop ---
for epoch in range(start_epoch, NUM_EPOCHS):

    model.train()
    epoch_train_loss_accum = 0.0 # Accumulator for average epoch loss
    interval_loss_accum = 0.0    # Accumulator for interval loss
    interval_steps = 0           # Counter for steps in current interval
    
    optimizer.zero_grad()

    # Wrap train_dataloader with tqdm for progress bar
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} train", unit="batch")

    for i, batch in enumerate(progress_bar):
        if batch is None:
            print(f"Warning: Skipped an empty batch at step {i} in epoch {epoch+1}")
            continue
        
        batch = {k: v.to(device) for k, v in batch.items()}

        with torch.amp.autocast(device_type=device.type, dtype=autocast_dtype, enabled=autocast_enabled):
            outputs = model(**batch)
            loss = outputs.loss
            if GRADIENT_ACCUMULATION_STEPS > 1:
                loss_scaled = loss / GRADIENT_ACCUMULATION_STEPS # Use a different var for scaled loss if needed
            else:
                loss_scaled = loss

        current_loss_item = loss.item() # Get the unscaled loss for logging
        if scaler.is_enabled():
            scaler.scale(loss_scaled).backward()
        else:
            loss_scaled.backward()
        
        epoch_train_loss_accum += current_loss_item # Accumulate unscaled loss for epoch average
        interval_loss_accum += current_loss_item
        interval_steps += 1

        if (i + 1) % GRADIENT_ACCUMULATION_STEPS == 0 or (i + 1) == len(train_dataloader):
            if scaler.is_enabled():
                # Optional: Clip gradients before optimizer step if using scaler
                # scaler.unscale_(optimizer) # Unscale for clipping
                # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # Example clip
                scaler.step(optimizer)
                scaler.update()
            else:
                # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # Example clip
                optimizer.step()
            optimizer.zero_grad()

        # ★★★ Log interval loss ★★★
        if (i + 1) % LOG_INTERVAL == 0 or (i + 1) == len(train_dataloader):
            if interval_steps > 0:
                avg_interval_loss = interval_loss_accum / interval_steps
                # Update tqdm progress bar description
                progress_bar.set_postfix({"loss": f"{avg_interval_loss:.4f}", "lr": f"{optimizer.param_groups[0]['lr']:.2e}"})
                # Or print to console:
                # print(f"  Epoch {epoch+1}, Step [{i+1}/{len(train_dataloader)}], Avg Interval Loss: {avg_interval_loss:.4f}, LR: {optimizer.param_groups[0]['lr']:.2e}")
                interval_loss_accum = 0.0 # Reset for next interval
                interval_steps = 0

    # --- Validation Phase ---
    model.eval()
    eval_loss_accum = 0.0
    with torch.no_grad():
        for batch in tqdm(valid_dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} valid", unit="batch"):
            if batch is None: continue
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.amp.autocast(device_type=device.type, dtype=autocast_dtype, enabled=autocast_enabled):
                outputs = model(**batch)
                eval_loss_accum += outputs.loss.item()

    # Calculate average losses for the epoch
    # avg_train_loss_epoch = epoch_train_loss_accum / len(train_dataloader) # Based on number of batches
    # More accurate if some batches were skipped or if last batch is partial:
    num_actual_train_batches = sum(1 for b in train_dataloader if b is not None) if len(train_dataloader) > 0 else 0
    avg_train_loss_epoch = epoch_train_loss_accum / num_actual_train_batches if num_actual_train_batches > 0 else 0.0
    
    num_actual_valid_batches = sum(1 for b in valid_dataloader if b is not None) if len(valid_dataloader) > 0 else 0
    avg_eval_loss = eval_loss_accum / num_actual_valid_batches if num_actual_valid_batches > 0 else 0.0
    
    current_lr = optimizer.param_groups[0]["lr"]
    tracking_information.append((avg_train_loss_epoch, avg_eval_loss, current_lr))
    print(f"\nEpoch {epoch+1} Summary: Avg Train Loss={avg_train_loss_epoch:.4f}, Avg Valid Loss={avg_eval_loss:.4f}, LR={current_lr:.2e}")

    if avg_eval_loss < min_eval_loss:
        print(f"Validation loss improved from {min_eval_loss:.4f} to {avg_eval_loss:.4f}. Saving model to {OUTPUT_MODEL_DIR}")
        min_eval_loss = avg_eval_loss
        os.makedirs(OUTPUT_MODEL_DIR, exist_ok=True)
        save_path = os.path.join(OUTPUT_MODEL_DIR, "checkpoint.pth")
        torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "scheduler_state_dict": scheduler.state_dict(),
            "scaler_state_dict": scaler.state_dict() if scaler.is_enabled() else None,
            "early_stop_counter": early_stop_counter,
            "min_eval_loss": min_eval_loss,
            "tracking_information": tracking_information
        }, save_path)
        
        processor.save_pretrained(OUTPUT_MODEL_DIR)
        model.save_pretrained(OUTPUT_MODEL_DIR)

        early_stop_counter = 0
    else:
        early_stop_counter += 1
        print(f"Validation loss did not improve. Early stopping counter: {early_stop_counter}/{PATIENCE}")
        if early_stop_counter >= PATIENCE:
            print("Early stopping triggered.")
            break
    scheduler.step()

# --- Save Tracking Information ---
tracking_file_dir = os.path.dirname(TRACKING_FILE)
if tracking_file_dir and not os.path.exists(tracking_file_dir):
    os.makedirs(tracking_file_dir, exist_ok=True)
with open(TRACKING_FILE, "wb") as f:
    pickle.dump(tracking_information, f)
print(f"Tracking information saved to {TRACKING_FILE}")
print("Training finished!")

Epoch 1/3 valid: 100%|██████████| 824/824 [06:08<00:00,  2.23batch/s]



Epoch 1 Summary: Avg Train Loss=7.5272, Avg Valid Loss=7.4681, LR=5.00e-05
Validation loss improved from inf to 7.4681. Saving model to /kaggle/working/blip-finetuned


Epoch 2/3 valid: 100%|██████████| 824/824 [06:08<00:00,  2.24batch/s]



Epoch 2 Summary: Avg Train Loss=7.4544, Avg Valid Loss=7.4598, LR=4.50e-05
Validation loss improved from 7.4681 to 7.4598. Saving model to /kaggle/working/blip-finetuned


Epoch 3/3 valid: 100%|██████████| 824/824 [06:10<00:00,  2.22batch/s]



Epoch 3 Summary: Avg Train Loss=7.4450, Avg Valid Loss=7.4558, LR=4.05e-05
Validation loss improved from 7.4598 to 7.4558. Saving model to /kaggle/working/blip-finetuned
Tracking information saved to tracking_information.pkl
Training finished!
